In [16]:
import os 

path = 'data/eopatches/test'
patches = os.listdir(path)
len(patches)

25

In [19]:
from tqdm import tqdm 
import rasterio 
from rasterio.profiles import DefaultGTiffProfile
from eolearn.core import EOPatch
from src.utils import generate_mask_with_mean_ndvi

submission_path = 'submission'
thresholds = (0.4, 0.6)
def generate_mask(patch):

    big_target = generate_mask_with_mean_ndvi(path, patch, thresholds)

    if big_target.sum() == 0:
        big_target[0,0] = 1

    # save
    file_path = f'{path}/{patch}'
    eopatch = EOPatch.load(file_path)
    # Gracias a @cayala
    tfm = rasterio.transform.from_bounds(*eopatch.bbox, eopatch.meta_info['size_x'], eopatch.meta_info['size_y'])
    tfm = rasterio.Affine(tfm.a/4, tfm.b, tfm.c, tfm.d, tfm.e/4, tfm.f)
    profile = DefaultGTiffProfile(count=1)
    profile.update(
        transform=tfm,
        width=2000,
        height=2000,
        crs='epsg:32633'
    )
    with rasterio.open(f'{submission_path}/{patch}.tif', 'w', **profile) as dst_dataset:
        dst_dataset.write(big_target, 1)
    

In [20]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor
import warnings

warnings.simplefilter("ignore")

num_cores = multiprocessing.cpu_count()
with ProcessPoolExecutor(max_workers=num_cores) as pool:
    with tqdm(total=len(patches)) as progress:
        futures = []

        for patch in patches:
            future = pool.submit(generate_mask, patch)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        scores = []
        for future in futures:
            result = future.result()
            scores.append(result)

100%|██████████| 25/25 [00:06<00:00,  3.73it/s]


In [22]:
!tar -C {submission_path} -zcvf submission.tar.gz .

./
./eopatch-23.tif
./eopatch-17.tif
./eopatch-21.tif
./eopatch-11.tif
./eopatch-18.tif
./eopatch-12.tif
./eopatch-25.tif
./eopatch-13.tif
./eopatch-08.tif
./eopatch-10.tif
./eopatch-22.tif
./eopatch-09.tif
./eopatch-20.tif
./eopatch-19.tif
./eopatch-02.tif
./eopatch-06.tif
./eopatch-14.tif
./eopatch-03.tif
./eopatch-05.tif
./eopatch-16.tif
./eopatch-04.tif
./eopatch-15.tif
./eopatch-01.tif
./eopatch-07.tif
./eopatch-24.tif


In [23]:
mean_ndvi.shape, mean_ndvi.dtype, mean_ndvi.max(), mean_ndvi.min()

((500, 500), dtype('float64'), nan, nan)

In [24]:
import matplotlib.pyplot as plt

plt.imshow(mean_ndvi)
plt.axis('off')
plt.show()

In [25]:
big_target.shape, big_target.dtype, big_target.max(), big_target.min()

((2000, 2000), dtype('bool'), True, False)

In [26]:
plt.imshow(big_target)
plt.axis('off')
plt.show()

In [27]:
sample = 'eopatch-22'
mask_ds = rasterio.open(f'submission/{sample}.tif')
mask_ds.crs, mask_ds.bounds

(CRS.from_epsg(32633),
 BoundingBox(left=569800.0, bottom=5940600.0, right=574800.0, top=5945600.0))

In [28]:
eopatch = EOPatch.load(f'{path}/{sample}')
print(eopatch.bbox)

569800.0,5940600.0,574800.0,5945600.0


In [29]:
mask = mask_ds.read(1)
mask.dtype, mask.shape, mask.min(), mask.max()

(dtype('uint8'), (2000, 2000), 0, 1)

In [30]:
plt.imshow(mask)
plt.axis('off')
plt.show()